In [140]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
import warnings
import math
import tensorflow as tf
warnings.simplefilter("ignore")
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import mean_squared_error
from sklearn.metrics import f1_score
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/fer2013.csv")
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
data_train = data[data['Usage']=='Training'].copy()
data_val   = data[data['Usage']=='PublicTest'].copy()
data_test  = data[data['Usage']=='PrivateTest'].copy()
print("train shape: {}, \nvalidation shape: {}, \ntest shape: {}".format(data_train.shape, data_val.shape, data_test.shape))

train shape: (28709, 3), 
validation shape: (3589, 3), 
test shape: (3589, 3)


In [ ]:
num_classes = 7 
width, height = 48, 48
num_epochs = 50
batch_size = 64
num_features = 64

In [ ]:
data['pixels'].dtype

dtype('O')

In [ ]:
n_samples = len(data)
n_samples_train = 28709
n_samples_test = 3589
n_samples_validation = 3589

y = data['emotion']
X = np.zeros((n_samples, width, height, 1))
for i in range(n_samples):
    X[i] = np.array([int(pixel) for pixel in data['pixels'][i].split()]).reshape(48,48,1)

#Training set   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

In [53]:
resnet = tf.keras.applications.resnet50.ResNet50(include_top=True, weights=None, input_shape=(48,48,1), classes=7, pooling='max', classifier_activation='softmax')

In [144]:
y_train = to_categorical(y_train)

In [161]:
resnet.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), 
              metrics=['accuracy'])

In [162]:
resnet.fit(X_train, y_train, epochs = 2)

Epoch 1/2
1010/1010 [==============================] - 129s 121ms/step - loss: 1.8450 - accuracy: 0.2989
Epoch 2/2
1010/1010 [==============================] - 122s 121ms/step - loss: 1.8016 - accuracy: 0.3080


In [165]:
pred = resnet.predict(X_test)

In [166]:
predictions = []
for p in pred:
  predictions.append(np.argmax(p))

In [167]:
f1_score(y_test, predictions, average='weighted')

0.13133265777174122

In [168]:
vggnet = tf.keras.applications.vgg19.VGG19(include_top=True, weights=None, input_shape=(48,48,1), classes=7, pooling='max', classifier_activation='softmax')

In [169]:
vggnet.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), 
              metrics=['accuracy'])
#compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['accuracy'])

In [178]:
vggnet.fit(X_train, y_train, epochs = 2)

Epoch 1/2
1010/1010 [==============================] - 110s 109ms/step - loss: 1.8118 - accuracy: 0.2515
Epoch 2/2
1010/1010 [==============================] - 110s 109ms/step - loss: 1.8114 - accuracy: 0.2515


In [179]:
print(X_test[0].shape)
pred = vggnet.predict(X_test)
predictions = []
for x in pred:
 predictions.append(np.argmax(x))

(48, 48, 1)


In [180]:
f1_score(y_test, predictions, average='weighted')

0.0940046961766779

In [174]:
def get_emotion(cl):
  k = []
  for i in range(len(y_test)):
    if y_test.iloc[i] == cl:
      k.append(X_test[i])
  return np.array(k)

angry = get_emotion(0)
fear = get_emotion(1)
sad = get_emotion(2)
neutral = get_emotion(3)
happy = get_emotion(4)
surprise = get_emotion(5)
disgust = get_emotion(6)

In [175]:
def check(model,input, cl):
  pred = []
  pred = model.predict(input)
  predictions = []
  for x in pred:
    predictions.append(np.argmax(x))
  y_k = []
  for i in range(len(input)):
    y_k.append(cl)
  return f1_score(y_k, predictions, average='weighted')

In [181]:
print(check(vggnet,angry, 0))
print(check(vggnet,fear, 1))
print(check(vggnet,sad, 2))
print(check(vggnet,neutral, 3))
print(check(vggnet,happy, 4))
print(check(vggnet,surprise, 5))
print(check(vggnet,disgust, 6))

0.0
0.0
0.0
1.0
0.0
0.0
0.0


In [177]:
print(check(resnet,angry, 0))
print(check(resnet,fear, 1))
print(check(resnet,sad, 2))
print(check(resnet,neutral, 3))
print(check(resnet,happy, 4))
print(check(resnet,surprise, 5))
print(check(resnet,disgust, 6))

0.801377726750861
0.0
0.0
0.5192143467122118
0.025764895330112725
0.0
0.090625
